# Gamma calculation

The PyMedPhys [gamma](../../ref/gamma.rst) allows to compute the gamma ($\gamma$) agreement index between a reference and an evaluation dose distribution in 1D, 2D and 3D.

In this tutorial, you will compute the pass rate using the PyMedPhys [gamma](../../ref/gamma.rst), and you will tweak its parameters to speed up the computation of the gamma index.

You will test the PyMedPhys [gamma](../../ref/gamma.rst) on two 3D (and 2D) dose distributions, and you will focus on two of the PyMedPhys [gamma](../../ref/gamma.rst) parameters, <code>max_gamma</code> and <code>random_subset</code>, which are particularly useful when performing pass rate calculations.

First, we import the required modules and download the DICOM RTDOSE data that you will use

In [ ]:
#%pip install pydicom pymedphys

import pydicom
import pymedphys
import numpy as np

reference_filepath = pymedphys.data_path("original_dose_beam_4.dcm") # reference (e.g., calculated) dose
evaluation_filepath = pymedphys.data_path("logfile_dose_beam_4.dcm") # evaluation (e.g., measured) dose

Both the reference and evaluation images are 3D dose distributions.

Then, we read the DICOM files using pydicom

In [ ]:
reference = pydicom.dcmread(reference_filepath)
evaluation = pydicom.dcmread(evaluation_filepath)

The positional arguments of the PyMedPhys [gamma](../../ref/gamma.rst) specify the axes and dose of the reference and evaluation images, as well as the dose difference and distance to agreement to perform the computation.

Now, we build the axes and get the dose values from the data we have read from the DICOM files

In [ ]:
axes_reference, dose_reference = pymedphys.dicom.zyx_and_dose_from_dataset(reference)
axes_evaluation, dose_evaluation = pymedphys.dicom.zyx_and_dose_from_dataset(evaluation)

<code>axes_reference</code> and <code>axes_evaluation</code> are tuples of numpy arrays, whose values define the (z, y, x) grid coordinates in the DICOM frame of reference.

Let's check the shape of the reference image axes

In [ ]:
z_ref, y_ref, x_ref = axes_reference

print(z_ref.shape, y_ref.shape, x_ref.shape)

That is, the 3D reference dose image is composed of 152 z-slices, 158 y-slices, and 254 x-slices.

<code>dose_reference</code> and <code>dose_evaluation</code> are matrices containing the actual dose values at each grid point.

Their shape will match that of the axes

In [ ]:
np.shape(dose_reference)

Let' have a look at the dose distributions above and below the z-slice where they disagree the most

In [ ]:
import matplotlib.pyplot as plt

dose_difference = dose_evaluation - dose_reference

max_diff = np.max(np.abs(dose_difference))

# get the z-slice with the maximum dose difference
z_max_diff, _, _ = np.unravel_index(np.argmax(np.abs(dose_difference), axis=None), dose_difference.shape)

# we consider 10 z-slices above and below the maximum dose difference
z_start = z_max_diff - 10
z_end = z_max_diff + 10

fig, axes = plt.subplots(figsize=(15,10), nrows=4, ncols=5, sharex=True, sharey=True)
ax = axes.ravel().tolist()
ax[0].invert_yaxis() # invert just once as y axes are shared

for i, dose_diff_slice in enumerate(dose_difference[z_start:z_end]):

    im = ax[i].contourf(x_ref, y_ref, dose_diff_slice, 100, cmap=plt.get_cmap("seismic"), vmin=-max_diff, vmax=max_diff)
    ax[i].set_title(f"Slice Z_{z_start + i}")
    
    if i >= 15:
        ax[i].set_xlabel("x (mm)")
    
    if i % 5 == 0:
        ax[i].set_ylabel("y (mm)")

fig.tight_layout()
cbar = fig.colorbar(im, ax=ax, label="[Dose Eval] - [Dose Ref] (Gy)", aspect=40)
cbar.formatter.set_powerlimits((0, 0))
plt.show()

As you can see, the largest dose differences are concentrated in small regions of the slices, which will end up in a fast gamma evaluation.

Now, we move to the actual calculation.

First of all, we define the acceptance criteria as 1% dose difference and 1mm distance

In [ ]:
dose_percent_threshold = 1
distance_mm_threshold = 1

By default, the 1% of <code>dose_percent_threshold</code> refers to the 1% of the maximum value of the reference dose, i.e.

In [ ]:
print(f"Maximum reference dose: {np.max(dose_reference): .2f} Gy")
print(f"Dose threshold: {np.max(dose_reference) * .01:.4f} Gy")

In order to see in detail the messages traced by the PyMedPhys [gamma](../../ref/gamma.rst), we import the logging module and set the logging level to DEBUG

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Now, we compute the gamma using the default settings of the PyMedPhys [gamma](../../ref/gamma.rst), and measure its computation time with <code>%%time</code>

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_reference, dose_reference, 
    axes_evaluation, dose_evaluation, 
    dose_percent_threshold, distance_mm_threshold)

Let's breakdown the log trace of the gamma function:

1. _Global normalisation set to 1.946_: the global normalization is set by default to the maximum value of the reference dose
2. _Global dose threshold set to 0.02 (1.00% of normalisation)_: the dose threshold value is (<code>dose_percent_threshold</code> * the global normalization / 100)
3. _Distance threshold set to [1]_: the <code>distance_mm_threshold</code>
4. _Lower dose cutoff set to 0.389 (20.0% of normalisation)_: the gamma is computed only for those points where the dose value is bigger than this cutoff (20% of the maximum reference dose)
...
5. _Current distance: 0.50 mm | Number of reference points remaining: 128_:
    - _Current distance_: the current distance between reference grid point and evaluated grid points. The computation stops once a distance is reached for which the value of the gamma cannot improve further
    - _Number of reference points remaining_: the number of points left to be evaluated in the reference dose grid
6. _Points tested per reference point: 348 | RAM split count: 1_:
    - _Points tested per reference point_: the number of points considered in the evaluation dose grid for the current reference point
    - _RAM split count_: in case the number of points to check for the gamma value doesn't fit into the designated RAM memory, the computation is split and performed in stages

The output stored in the <code>gamma</code> variable is a matrix containing the gamma index value for each point of the reference dose

In [ ]:
np.shape(gamma)

Let's check the distribution of the gamma values to get a sense of the function output

In [ ]:
import matplotlib.pyplot as plt
valid_gamma = gamma[~np.isnan(gamma)]

out = plt.hist(valid_gamma, bins=20, density=True)
plt.xlabel("gamma index")
_ = plt.ylabel("probability density")

Once the gamma has been computed, one usually considers the passing rate to have a final evaluation of the agreement between the reference and evaluated dose distributions.

The passing rate is defined as the fraction of points having $\gamma\le 1$. A threshold is chosen (e.g., 95%) and the test is passed if the passing rate exceeds such threshold.

We check if in this case the test is passed (probably you already know the answer looking at the histogram above)

In [ ]:
print(f"Pass rate(\u03B3<=1): {len(valid_gamma[valid_gamma <= 1]) / len(valid_gamma) * 100}%")

Indeed, all points have $\gamma\le 1$, thus the pass rate is 100%.

As expected, the computation of the gamma was quite fast because most of the grid points have zero dose difference

In [ ]:
a = dose_difference[dose_difference == 0]
b = dose_difference[dose_difference != 0]

print(f"Fraction of grid points where [Dose Eval] = [Dose Ref]: {np.sum(dose_difference == 0) / len(dose_difference.flat) * 100:.1f}%")

In practice, one might need to perform computations of the gamma for several dose comparisons, which could result in a time consuming operation.

In case we need only the final pass rate, it is desirable to save as much time as possible.

The PyMedPhys gamma function accepts optional parameters that can speed up the calculation in such cases:
- <code>max_gamma</code>: stop the calculation when a certain value of gamma is reached around the reference point
- <code>random_subset</code>: use only a random subset of the reference dose grid (range: 0-x*y*z)

Let's see how these parameters affect the gamma calculation time.

For simplicity, we restrict ourself on the z-slice with the highest disagreement.

We also add a small shift to the evaluation dose distribution to obtain a worse gamma agreement which will make the computation slower

In [ ]:
# add 1% of the evaluation dose standard deviation where the evaluation dose in non zero
dose_evaluation_Z = np.where(
    dose_evaluation[z_max_diff] != 0,
    dose_evaluation[z_max_diff] + .1 * np.std(dose_evaluation[z_max_diff]),
    dose_evaluation[z_max_diff]
)
dose_reference_Z = np.array(dose_reference[z_max_diff])

# keep only the y, x axes
axes_reference_subset = axes_reference[1:]
axes_evaluation_subset = axes_evaluation[1:]

In [ ]:
# increase the logging level to silence the traces of the PyMedPhys gamma
logger.setLevel(logging.ERROR)

Furthermore, let's say we are now interested in evaluating all those points above 1% of the maximum value of the reference dose (instead of the default 20%): we pass the <code>lower_percent_dose_cutoff</code> to the PyMedPhys [gamma](../../ref/gamma.rst)

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_reference_subset, dose_reference_Z, 
    axes_evaluation_subset, dose_evaluation_Z, 
    dose_percent_threshold, distance_mm_threshold,
    lower_percent_dose_cutoff=1 # 1% lower threshold
    )

In [ ]:
valid_gamma = gamma[~np.isnan(gamma)]

out = plt.hist(valid_gamma, bins=20, density=True)
plt.xlabel("gamma index")
plt.ylabel("probability density")
title = plt.title(f"Gamma passing rate: {np.sum(valid_gamma <= 1) / len(valid_gamma) * 100:.1f}%")

If one is interested in evaluating the passing rate, it is sufficient to check whether the gamma index of a reference point is above or below 1.

The <code>max_gamma</code> parameter of the PyMedPhys [gamma](../../ref/gamma.rst) can be used for this purpose, making the computation faster

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_reference_subset, dose_reference_Z, 
    axes_evaluation_subset, dose_evaluation_Z, 
    dose_percent_threshold, distance_mm_threshold,
    lower_percent_dose_cutoff=1, # 1% lower threshold
    max_gamma=1.1 # stop when gamma > 1.1
    )

In [ ]:
valid_gamma = gamma[~np.isnan(gamma)]

out = plt.hist(valid_gamma, bins=20, density=True)
plt.xlabel("gamma index")
plt.ylabel("probability density")
title = plt.title(f"Gamma passing rate: {np.sum(valid_gamma <= 1) / len(valid_gamma) * 100:.1f}%")

Notice how the gamma passing rate value (shown in the plot title) did not change. Of course now all the points with $\gamma > 1.1$ are now collapsed to the 1.1 bin.

Finally, let's see the effect of the second parameter, i.e. <code>random_subset</code>, that can speed up the calculation of the PyMedPhys [gamma](../../ref/gamma.rst)

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_reference_subset, dose_reference_Z, 
    axes_evaluation_subset, dose_evaluation_Z, 
    dose_percent_threshold, distance_mm_threshold,
    lower_percent_dose_cutoff=1,
    random_subset=int(len(dose_reference_Z.flat) // 100) # sample only one 100th of the grid points
    )

The $\gamma$ index distribution should look something like the following plot

In [ ]:
valid_gamma = gamma[~np.isnan(gamma)]

out = plt.hist(valid_gamma, bins=20, density=True)
plt.xlabel("gamma index")
plt.ylabel("probability density")
title = plt.title(f"Gamma passing rate: {np.sum(valid_gamma <= 1) / len(valid_gamma) * 100:.1f}%")

As you can see the computation time decreased and, despite the the PyMedPhys [gamma](../../ref/gamma.rst) sampled one 100th of the points, the pass rate changed only by a few percent. Therefore, you can use <code>random_subset</code> to get a sense of what could be the pass rate of the dose distributions you are assessing.

To conclude, you have performed several computations of the gamma index and the pass rate of 3D and 2D dose distributions using the PyMedPhys [gamma](../../ref/gamma.rst).

Moreover, you have obtained fast pass rate computations by tweaking the <code>max_gamma</code> and <code>random_subset</code> parameters of the PyMedPhys [gamma](../../ref/gamma.rst), independently.

Of course, both <code>max_gamma</code> and <code>random_subset</code> can be used at the same time. You are now ready to test them by yourself!